In [ ]:
Q1. What is Gradient Boosting Regression?

Gradient boosting regression trees are based on the idea of an ensemble method derived from a decision tree.
The decision tree uses a tree structure. Starting from tree root, branching according to the conditions and
heading toward the leaves, the goal leaf is the prediction result.


In [ ]:
Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy. Use a simple 
regression problem as an example and train the model on a small dataset. Evaluate the model's performance
using metrics such as mean squared error and R-squared.

In [ ]:
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,make_scorer
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

class GradientBoostTreeRegressor(object):
    #initialiser
    def __init__(self, n_elements : int = 100, max_depth : int = 1) -> None:
        self.max_depth     = max_depth
        self.n_elements    = n_elements
        self.f             = []
        self.regions       = []
        self.gammas        = []
        self.mean_loss     = []
        self.e0            = 0
        
    #destructor
    def __del__(self) -> None:
        del self.max_depth
        del self.n_elements
        del self.f
        del self.regions
        del self.gammas
        del self.mean_loss
        del self.e0

#private function to group data points & compute gamma parameters
    def __compute_gammas(self, yp : np.array, y_train : np.array, e : np.array) -> Tuple[np.array,Dict]:
        #initialise global gamma array
        gamma_jm = np.zeros((y_train.shape[0]))
        #iterate through each unique predicted value/region
        regions = np.unique(yp)
        gamma   = {}
        for r in regions:
            #compute index for r
            idx = yp == r
            #isolate relevant data points
            e_r = e[idx]
            y_r = y_train[idx]
            #compute the optimal gamma parameters for region r
            gamma_r = np.median(y_r - e_r)
            #populate the global gamma array
            gamma_jm[idx] = gamma_r
            #set the unique region <-> gamma pairs
            gamma[r] = gamma_r
        #append the regions to internal storage
        self.regions.append(regions)
        #return
        return((gamma_jm,gamma))

#public function to train the ensemble
    def fit(self, X_train : np.array, y_train : np.array) -> None:
        #reset the internal class members
        self.f             = []
        self.regions       = []
        self.model_weights = []
        self.mean_loss     = []
        #initialise the ensemble & store initialisation
        e0      = np.median(y_train)
        self.e0 = np.copy(e0)
        e       = np.ones(y_train.shape[0]) * e0
        #loop through the specified number of iterations in the ensemble
        for _ in range(self.n_elements):
            #store mae loss
            self.mean_loss.append(np.mean(np.abs(y_train - e)))
            #compute the gradients of our loss function
            g = np.sign(y_train - e)
            #initialise a weak learner & train
            model = DecisionTreeRegressor(max_depth=self.max_depth)
            model.fit(X_train,g)
            #compute optimal gamma coefficients
            yp             = model.predict(X_train)
            gamma_jm,gamma = self.__compute_gammas(yp,y_train,e)
            #update the ensemble
            e += gamma_jm
            #store trained ensemble elements
            self.f.append(model)
            self.gammas.append(gamma)
            

#public function to generate predictions
    def predict(self, X_test : np.array) -> np.array:
        #initialise predictions
        y_pred = np.ones(X_test.shape[0]) * np.copy(self.e0)
        #cycle through each element in the ensemble
        for model,gamma,regions in zip(self.f,self.gammas,self.regions):
            #produce predictions using model
            y = model.predict(X_test)
            #cycle through each unique leaf node for model m
            for r in regions:
                #updates for region r
                idx = y == r
                y_pred[idx] += gamma[r] 
        #return predictions
        return(y_pred)
           
    #public function to return mean training loss
    def get_loss(self) -> List:
        return(self.mean_loss)
 
    #public function to return model parameters
    def get_params(self, deep : bool = False) -> Dict:
        return {'n_elements':self.n_elements,
                'max_depth':self.max_depth}


fX,sy = load_diabetes(return_X_y=True,as_frame=True)
rgr = GradientBoostTreeRegressor(n_elements=100)
rgr.fit(dfX.values,sy.values)

loss1 = rgr.get_loss()
rgr = GradientBoostTreeRegressor(n_elements=100, max_depth=2)
rgr.fit(dfX.values,sy.values)

loss2 = rgr.get_loss()
rgr = GradientBoostTreeRegressor(n_elements=100, max_depth=3)
rgr.fit(dfX.values,sy.values)

loss3 = rgr.get_loss()
rgr = GradientBoostTreeRegressor(n_elements=100, max_depth=4)
rgr.fit(dfX.values,sy.values)

loss4 = rgr.get_loss()

plt.plot(loss1,label='max_depth=1')
plt.plot(loss2,label='max_depth=2')
plt.plot(loss3,label='max_depth=3')
plt.plot(loss4,label='max_depth=4')
plt.title('Training Loss by Boosting Iteration')
plt.xlabel('Number of Component Trees')
plt.ylabel('MAE Loss')
plt.legend()
plt.show()


scoring_metrics = {'mse' : make_scorer(mean_squared_error), 
                   'mae': make_scorer(mean_absolute_error)}


rgr = GradientBoostTreeRegressor(n_elements=20,max_depth=1)
#cross validate
dcScores = cross_validate(rgr,dfX.values,sy.values,cv=10,scoring=scoring_metrics)
#report results
print('Mean MSE: %.2f' % np.mean(dcScores['test_mse']))
print('Mean MAE: %.2f' % np.mean(dcScores['test_mae']))



In [ ]:
Q3. Experiment with different hyperparameters such as learning rate, number of trees, and tree depth
to optimise the performance of the model. Use grid search or random search to find the best hyperparameters.

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('dataset.csv')
X = df.drop('target', axis = 1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
rfc = RandomForestClassifier()
forest_params = [{'max_depth': list(range(10, 15)), 'max_features': list(range(0,14))}]
clf = GridSearchCV(rfc, forest_params, cv = 10, scoring='accuracy')
clf.fit(X_train, y_train)
print(clf.best_params_)
print(clf.best_score_)


{'max_depth': 12, 'max_features': 1}
0.8582251082251082


In [ ]:
Q4. What is a weak learner in Gradient Boosting?

The term Weak Learner refers to simple models that do only slightly better than random chance.
Boosting algorithms start with a single weak learner (tree methods are overwhelmingly used here),
but technically, any model will do.


In [ ]:
Q5. What is the intuition behind the Gradient Boosting algorithm?

Gradient boosting is a type of machine learning boosting. It relies on the intuition that the best possible 
next model, when combined with previous models, minimizes the overall prediction error. The key idea is to 
set the target outcomes for this next model in order to minimize the error.


In [ ]:
Q6. How does Gradient Boosting algorithm build an ensemble of weak learners?

The gradient-boosting regressor works by iteratively building an ensemble of weak learners, where each
subsequent weak learner is trained to correct the mistakes made by the previous ones. The predictions 
from all weak learners are combined to make the final prediction.


In [ ]:
Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting algorithm?
The steps involved  in  constructing  the mathematical  intuition  of Gradient Boosting  algorithm
 

Step1:
It is the calculation of the average value which is defined as a constant value. We calculate the constant
value or the first predicted Salary value by applying the concept of minimizing the difference between actual and
predicted results i.e. Loss function. Mathematically minimizing anything means differentiation at that point is zero. 
Hence in the below image we
 

Now putting back the values of y for each record we can calculate the predicted value or average value for
the entire dataset.
 
Step 2(a):
For Ῡ = F₀( x ) = predicted value we need to now calculate the residual or difference of predicted value w.r.t y.
To calculate residual we use the Decision Tree which is represented by m.
rᵢₘ = residual of record I for decision tree m.
 
Step 2(b):We now construct decision tree 1 using the residual as outputs and the features x as inputs.
Step 2(c):For each actual residual output which is obtained in the above step, we will now calculate the predicted
output Ῡ.
We already have the predicted value of the model1 which is defined as the average value in step-1.
To obtain the residual predicted output in the Decision Tree we are using the concept of loss function of step-1.
j = leaf node in decision tree
yᵢ = actual output
Ῡ = predicted output
Fₘ-₁( x ) = previous model output
 
Step-2(d)
After obtaining the predicted value of the Decision tree we combine it with the previous model value. 
The loop will keep on repeating itself until the predicted value is equal to the actual value.
 

